In [1]:
import torch
from modules import SConv2d, SLinear, SModel, SReLU
from models import SCrossEntropyLoss

In [4]:
def train_one_epoch():
    
    return loss.item()

In [17]:
fc1 = SLinear(3,3,bias=False)
# fc1.op.weight.data = torch.Tensor(list(range(1,10))).view(fc1.op.weight.shape) * 0.1
# fc1.op.bias.data = torch.zeros_like(fc1.op.bias)
fc2 = SLinear(3,3,bias=False)
# fc2.op.weight.data = torch.Tensor(list(range(9,0,-1))).view(fc2.op.weight.shape) * 0.1
# fc2.op.weight.data[0,0] = 2
# fc2.op.bias.data = torch.zeros_like(fc2.op.bias)
relu = SReLU()
criteria = SCrossEntropyLoss()
optimizer = torch.optim.SGD(list(fc1.parameters()) + list(fc2.parameters()), 1e-1)

for _ in range(1000):
    optimizer.zero_grad()
    x = torch.Tensor([[1,2,3]])
    xS = torch.zeros_like(x)
    labels = torch.LongTensor([0])

    x1, xS1 = fc1(x, xS)
    x1.retain_grad()
    xS1.retain_grad()
    x2, xS2 = relu(x1, xS1)
    x2.retain_grad()
    xS2.retain_grad()
    x3, xS3 = fc2(x2, xS2)
    x3.retain_grad()
    xS3.retain_grad()

    loss = criteria(x3, xS3, labels)
    loss.backward()
    optimizer.step()

optimizer = torch.optim.SGD(list(fc1.parameters()) + list(fc2.parameters()), 0)
optimizer.zero_grad()
x = torch.Tensor([[1,2,3]])
xS = torch.zeros_like(x)
labels = torch.LongTensor([0])

x1, xS1 = fc1(x, xS)
x1.retain_grad()
xS1.retain_grad()
x2, xS2 = relu(x1, xS1)
x2.retain_grad()
xS2.retain_grad()
x3, xS3 = fc2(x2, xS2)
x3.retain_grad()
xS3.retain_grad()

loss = criteria(x3, xS3, labels)
loss.backward()
optimizer.step()
print(loss)

tensor(0.0001, grad_fn=<SCrossEntropyLossFunctionBackward>)


In [3]:
# x3SG = torch.Tensor([[0.0123928,-0.0117341,-0.000658689,-0.0117341,0.011742,-7.92681E-6,-0.000658689,-7.92681E-6,0.000666616]]).view(1,3,3)
# x1SG = torch.Tensor([[0.0249031,0.00560088,0.00560088,0.00560088,0.00129391,0.00129391,0.00560088,0.00129391,0.00129391]]).view(1,3,3)

In [18]:
fc1.weightS.grad

tensor([[2.8934e-05, 1.1574e-04, 2.6041e-04],
        [2.3459e-04, 9.3835e-04, 2.1113e-03],
        [3.8065e-05, 1.5226e-04, 3.4258e-04]])

In [19]:
def CECross(output):
    BS = output.shape[0]
    e3 = (output - output.max()).exp()
    e3_sum = e3.sum()
    ratio = (e3 / e3_sum).view(BS,1,-1)

    return (torch.diag_embed(ratio.view(BS,-1),0,1) - ratio.swapaxes(1,2).bmm(ratio))
x3SG = CECross(x3)

In [20]:
x1SG = torch.zeros(1,3,3)
for i in range(3):
    for j in range(3):
        for k in range(3):
            for m in range(3):
                x1SG[:,i,j] += x3SG[:,k,m] * fc2.op.weight[k,i] * fc2.op.weight[m,j]

In [21]:
def crossSecond(IN, outS):
    BS = IN.shape[0]
    IN = IN.view(BS,1,-1)
    return IN.swapaxes(1,2).bmm(IN).view(BS,1,-1).swapaxes(1,2).bmm(outS.view(BS,1,-1))
fc1WSG = crossSecond(x,  x1SG)
fc2WSG = crossSecond(x2, x3SG)

In [22]:
fc1WSG

tensor([[[5.1460e-05, 1.3437e-04, 3.7396e-05, 1.3437e-04, 4.1264e-04,
          8.2717e-05, 3.7396e-05, 8.2717e-05, 3.0779e-05],
         [1.0292e-04, 2.6873e-04, 7.4792e-05, 2.6873e-04, 8.2527e-04,
          1.6543e-04, 7.4792e-05, 1.6543e-04, 6.1558e-05],
         [1.5438e-04, 4.0310e-04, 1.1219e-04, 4.0310e-04, 1.2379e-03,
          2.4815e-04, 1.1219e-04, 2.4815e-04, 9.2337e-05],
         [1.0292e-04, 2.6873e-04, 7.4792e-05, 2.6873e-04, 8.2527e-04,
          1.6543e-04, 7.4792e-05, 1.6543e-04, 6.1558e-05],
         [2.0584e-04, 5.3746e-04, 1.4958e-04, 5.3746e-04, 1.6505e-03,
          3.3087e-04, 1.4958e-04, 3.3087e-04, 1.2312e-04],
         [3.0876e-04, 8.0620e-04, 2.2438e-04, 8.0620e-04, 2.4758e-03,
          4.9630e-04, 2.2438e-04, 4.9630e-04, 1.8467e-04],
         [1.5438e-04, 4.0310e-04, 1.1219e-04, 4.0310e-04, 1.2379e-03,
          2.4815e-04, 1.1219e-04, 2.4815e-04, 9.2337e-05],
         [3.0876e-04, 8.0620e-04, 2.2438e-04, 8.0620e-04, 2.4758e-03,
          4.9630e-04, 2.243

In [28]:
fc2WSG

tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 3.2674e-03, -2.2173e-03, -1.0508e-03, -2.2173e-03,  2.2174e-03,
          -8.9876e-08, -1.0508e-03, -8.9876e-08,  1.0509e-03],
         [ 6.9283e-04, -4.7016e-04, -2.2282e-04, -4.7016e-04,  4.7018e-04,
          -1.9058e-08, -2.2282e-04, -1.9058e-08,  2.2284e-04],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 6.9283e-04, -4.7016e-04

In [32]:
fc1WSG.view(9,9).diag().view(-1).sort()[1]

tensor([0, 6, 2, 1, 3, 8, 5, 7, 4])

In [33]:
fc1.weightS.grad.view(-1).sort()[1]

tensor([0, 6, 1, 7, 3, 2, 8, 4, 5])